# Migração de Usuários de "tkl-odoo15-jcafb25-vm" para "tkl-odoo16-vm-18"

## Leitura das Credenciais

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data/jcafb_2025.db')

hosts = pd.read_sql_query('SELECT * FROM host', conn, index_col='name')
# display(hosts.head())

# conn.commit()
conn.close()

creds = lambda index, col_name : hosts.at[index, col_name]


## Accesso ao Servidor "tkl-odoo15-jcafb25-vm" (host remoto)

In [2]:
import host

rhost_name = 'tkl-odoo15-jcafb25-vm'

rhost = host.host_login(creds(rhost_name, 'server'),
                        creds(rhost_name, 'dbname'),
                        creds(rhost_name, 'user'),
                        creds(rhost_name, 'user_pw'))
rhost

INFO:host:--> host_login https://192.168.25.210 clvhealth_jcafb_2025_15
INFO:host:--> uid 2
INFO:host:--> sock <ServerProxy for 192.168.25.210/xmlrpc/2/object>
INFO:host:--> [01] Login Ok. Administrator 
INFO:host:--> {'id': 2, 'name': 'Administrator', 'parent_id': False}


(2, <ServerProxy for 192.168.25.210/xmlrpc/2/object>, '[01] Login Ok.')

## Accesso ao Servidor "tkl-odoo16-vm-18" (host local)

In [3]:
lhost_name = 'tkl-odoo16-vm-18'

lhost = host.host_login(creds(lhost_name, 'server'),
                        creds(lhost_name, 'dbname'),
                        creds(lhost_name, 'user'),
                        creds(lhost_name, 'user_pw'))

lhost

INFO:host:--> host_login https://192.168.25.212 clvhealth_jcafb_16
INFO:host:--> uid 2
INFO:host:--> sock <ServerProxy for 192.168.25.212/xmlrpc/2/object>
INFO:host:--> [01] Login Ok. Administrator 
INFO:host:--> {'id': 2, 'name': 'Administrator', 'parent_id': False}


(2, <ServerProxy for 192.168.25.212/xmlrpc/2/object>, '[01] Login Ok.')

## Ler as informações de res_users do host remoto

In [4]:
# import res_users

# res_users.res_user_migrate(rhost[1], rhost[0], creds(rhost_name, 'user_pw'), creds(rhost_name, 'dbname'),
#                            lhost[1], lhost[0], creds(lhost_name, 'user_pw'), creds(lhost_name, 'dbname'))

In [5]:
import logging

logging.basicConfig(level=logging.DEBUG)
_logger = logging.getLogger(__name__)



In [6]:
def get_data(sock, uid, admin_user_pw, dbname,
             model_name, model_fields):

    count = 0
    execute = True
    while execute:
        try:
            model_objects = sock.execute(dbname, uid, admin_user_pw,
                                         model_name, 'search_read',
                                         [],
                                         model_fields)
            execute = False
        except Exception as err:
            count = count + 1
            if count > 10:
                raise err
            else:
               _logger.info(u'%s %s\n', '--> Exception:', err)

    _logger.info(u'%s %s\n', '--> model_objects', len(model_objects))

    return model_objects


In [7]:
res_users_fields = ['id', 'name', 'partner_id', 'company_id', 'parent_id', 'tz', 'lang', 'country_id',
                    'login', 'password', 'groups_id', 'active', 'image_1920']


res_users_objects = get_data(rhost[1], rhost[0], creds(rhost_name, 'user_pw'), creds(rhost_name, 'dbname'),
                             'res.users', res_users_fields)

count = 0
for res_users_object in res_users_objects:

    if count < 5:
        print(res_users_object['id'],
              res_users_object['name'],
              res_users_object['partner_id'],
              res_users_object['company_id'],
              res_users_object['parent_id'],
              res_users_object['tz'],
              res_users_object['lang'],
              res_users_object['country_id'],
              res_users_object['login'],
              res_users_object['password'],
              res_users_object['groups_id'],
              res_users_object['active'],
              # res_users_object['image_1920']
             )
        count = count + 1

res_partner_fields = ['id', 'name', 'type']

res_partner_objects = get_data(rhost[1], rhost[0], creds(rhost_name, 'user_pw'), creds(rhost_name, 'dbname'),
                               'res.partner', res_partner_fields)

count = 0
for res_partner_object in res_partner_objects:

    if count < 10:
        print(res_partner_object['id'],
              res_partner_object['name'],
              res_partner_object['type'],
             )
        count = count + 1

res_company_fields = ['id', 'name']

res_company_objects = get_data(rhost[1], rhost[0], creds(rhost_name, 'user_pw'), creds(rhost_name, 'dbname'),
                               'res.company', res_company_fields)

count = 0
for res_company_object in res_company_objects:

    if count < 10:
        print(res_company_object['id'],
              res_company_object['name'],
             )
        count = count + 1


INFO:__main__:--> model_objects 259



2 Administrator [3, 'Administrator'] [1, 'CLVhealth-JCAFB'] False America/Sao_Paulo pt_BR False admin  [2, 11, 7, 14, 16, 68, 72, 8, 1, 20, 12, 21, 66, 39, 63, 77, 45, 30, 33, 70, 54, 83, 51, 27, 74, 60, 48, 36, 80, 42, 57, 13, 19, 3, 22, 67, 40, 64, 78, 46, 31, 34, 71, 55, 84, 52, 28, 75, 61, 49, 37, 81, 43, 58, 18, 6, 15, 17, 65, 38, 62, 76, 44, 29, 32, 69, 53, 82, 50, 26, 73, 59, 47, 35, 79, 41, 56] True
8 Alex Monteiro Magalhães [10, 'CLVhealth-JCAFB, Alex Monteiro Magalhães'] [1, 'CLVhealth-JCAFB'] [1, 'CLVhealth-JCAFB'] America/Sao_Paulo pt_BR False alex.magalhaes@gmail.com  [11, 7, 14, 16, 8, 1, 12, 13, 6, 15] True
9 Alexandre Bakanovas [11, 'CLVhealth-JCAFB, Alexandre Bakanovas'] [1, 'CLVhealth-JCAFB'] [1, 'CLVhealth-JCAFB'] America/Sao_Paulo pt_BR False alexandre.bakanovas@usp.br  [11, 7, 14, 16, 8, 1, 12, 13, 6, 15] True
10 Alice Herminia Serpentino [12, 'CLVhealth-JCAFB, Alice Herminia Serpentino'] [1, 'CLVhealth-JCAFB'] [1, 'CLVhealth-JCAFB'] America/Sao_Paulo pt_BR False a

INFO:__main__:--> model_objects 1947



3931 Abel Lucas Bertoni clv.patient_rec
3481 Abel Lucas Bertoni clv.patient
3932 Adalberto de Sousa clv.patient_rec
3496 Adalberto de Sousa clv.patient
3933 Adelaide Francisco Machado clv.patient_rec
3724 Adelaide Francisco Machado clv.patient
3934 Adelia Aparecida Fontana clv.patient_rec
3671 Adelia Aparecida Fontana clv.patient
3935 Ademar Antonio Dagoano clv.patient_rec
3504 Ademar Antonio Dagoano clv.patient


INFO:__main__:--> model_objects 1



1 CLVhealth-JCAFB


In [8]:
res_users_fields = ['id', 'name', 'partner_id', 'company_id', 'parent_id', 'tz', 'lang', 'country_id',
                    'login', 'password', 'groups_id', 'active', 'image_1920']


res_users_objects = get_data(lhost[1], lhost[0], creds(lhost_name, 'user_pw'), creds(lhost_name, 'dbname'),
                             'res.users', res_users_fields)

count = 0
for res_users_object in res_users_objects:

    if count < 5:
        print(res_users_object['id'],
              res_users_object['name'],
              res_users_object['partner_id'],
              res_users_object['company_id'],
              res_users_object['parent_id'],
              res_users_object['tz'],
              res_users_object['lang'],
              res_users_object['country_id'],
              res_users_object['login'],
              res_users_object['password'],
              res_users_object['groups_id'],
              res_users_object['active'],
              # res_users_object['image_1920']
             )
        count = count + 1

res_partner_fields = ['id', 'name', 'type']

res_partner_objects = get_data(lhost[1], lhost[0], creds(lhost_name, 'user_pw'), creds(lhost_name, 'dbname'),
                               'res.partner', res_partner_fields)

count = 0
for res_partner_object in res_partner_objects:

    if count < 10:
        print(res_partner_object['id'],
              res_partner_object['name'],
              res_partner_object['type'],
             )
        count = count + 1

res_company_fields = ['id', 'name']

res_company_objects = get_data(lhost[1], lhost[0], creds(lhost_name, 'user_pw'), creds(lhost_name, 'dbname'),
                               'res.company', res_company_fields)

count = 0
for res_company_object in res_company_objects:

    if count < 10:
        print(res_company_object['id'],
              res_company_object['name'],
             )
        count = count + 1


INFO:__main__:--> model_objects 2



2 Administrator [3, 'Administrator'] [1, 'CLVhealth-JCAFB'] False America/Sao_Paulo pt_BR False admin  [2, 12, 8, 17, 23, 19, 3, 9, 1, 13, 6, 16, 4, 7, 18, 20, 22] True
6 Data Administrator [7, 'CLVhealth-JCAFB, Data Administrator'] [1, 'CLVhealth-JCAFB'] [1, 'CLVhealth-JCAFB'] America/Sao_Paulo pt_BR [31, 'Brazil'] data.admin  [12, 8, 19, 23, 17, 9, 1, 13, 6, 16, 7, 20, 22, 18] True


INFO:__main__:--> model_objects 4

INFO:__main__:--> model_objects 1



8 Administrator private
3 Administrator contact
1 CLVhealth-JCAFB contact
7 Data Administrator contact
1 CLVhealth-JCAFB
